# Content:

- parse IFC models into graph database
- run Diff
- generate patch
- apply patch

## Parse BIM models into graph database

In [1]:
import time
from typing import List

from IfcGraphInterface.Ifc2GraphTranslator import IFCGraphGenerator
from neo4j_middleware.neo4jConnector import Neo4jConnector

print('[INFO] Parsing Ifc StepP21 model to Neo4j.... \n')

# connect to neo4j
connector = Neo4jConnector()
connector.connect_driver()

# paths to IFC models
model_paths: List[str] = [
    './00_sampleData/IFC_stepP21/Fahrzeughalle/Fahrzeughalle.ifc',
    './00_sampleData/IFC_stepP21/Fahrzeughalle/Fahrzeughalle_updt.ifc']

print('[INFO] Starting to generate graphs...')
amount = len(model_paths)
start = time.perf_counter()

model_labels = []

for idx, path in enumerate(model_paths):
    # parse model
    graphGenerator = IFCGraphGenerator(connector, path, None)
    print('Generating Graph %d/%d' % (idx + 1, amount))
    label = graphGenerator.generateGraph()
    model_labels.append(label)

finish = time.perf_counter()
print('\n[INFO] 100% done. Graphs generated. Finished in {} seconds.'.format(round(finish - start, 2)))
# disconnect from database
connector.disconnect_driver()

label_init = model_labels[0]
label_updated = model_labels[1]

del idx, finish, start, model_labels, label, graphGenerator

[INFO] Parsing Ifc StepP21 model to Neo4j.... 

Initialized new Connector instance.
[INFO] Starting to generate graphs...
Generating Graph 1/2
[IFC_P21 > ts20220510T140147 < ]: Generating graph... 
[####################################################################################################] 99% done.
[####################################################################################################] 99% done.
[####################################################################################################] 99% done.
[####################################################################################################] 99% done.
[####################################################################################################] 99% done.
[####################################################################################################] 99% done.
[####################################################################################################] 99% done.
[##########

## Run diff

runs the depth-first traversal to compare both model graphs and connects equivalent nodes using `EQUIVALENT_TO` edges

In [2]:
from neo4jGraphDiff.GraphDiff import GraphDiff
from neo4j_middleware.ResponseParser.NodeItem import NodeItem
from neo4j_middleware.neo4jConnector import Neo4jConnector
import jsonpickle

print('Run Diff ... \n')

# connect to neo4j
connector = Neo4jConnector()
connector.connect_driver()

label_init = "ts20220510T140147"
label_updated = "ts20220510T140041"

# get topmost entry nodes
raw_init = connector.run_cypher_statement(
    """
    MATCH (n:PrimaryNode:{} {{EntityType: "IfcProject"}})
    RETURN ID(n), n.EntityType, PROPERTIES(n), LABELS(n)
    """.format(label_init))
raw_updated = connector.run_cypher_statement(
    """
    MATCH (n:PrimaryNode:{} {{EntityType: "IfcProject"}})
    RETURN ID(n), n.EntityType, PROPERTIES(n), LABELS(n)
    """.format(label_updated))

# entry_init: NodeItem = NodeItem.from_neo4j_response_wou_rel(raw_init)[0]
# entry_updated: NodeItem = NodeItem.from_neo4j_response_wou_rel(raw_updated)[0]
entry_init: NodeItem = NodeItem.from_neo4j_response(raw_init, False)[0]
entry_updated: NodeItem = NodeItem.from_neo4j_response(raw_updated, False)[0]

# run diff
pDiff = GraphDiff(connector=connector, ts_init=label_init, ts_updated=label_updated)
delta = pDiff.diff_subgraphs(entry_init, entry_updated)

# Create EQUIVALENT_TO relationships to mark all nodePairs that are matched
print('[INFO] building EQUIVALENT_TO edges ... ')
pDiff.build_equivalent_to_edges()
print('[INFO] building EQUIVALENT_TO edges: DONE. ')

# store result
print('[INFO] saving delta ... ')
f = open('GraphDelta_init{}-updt{}.json'.format(label_init, label_updated), 'w')
f.write(jsonpickle.dumps(delta))
f.close()
print('[INFO] saving delta: DONE. ')


connector.disconnect_driver()
del raw_init, raw_updated, entry_init, entry_updated, GraphDiff, f

Run Diff ... 

Initialized new Connector instance.
[DIFF] Running subgraph Diff under PrimaryNodes 11503 and 34559
[DIFF] Running subgraph Diff under PrimaryNodes 11500 and 34556
[DIFF] Running subgraph Diff under PrimaryNodes 297 and 11824
[DIFF] Running subgraph Diff under PrimaryNodes 909 and 23965
[DIFF] Running subgraph Diff under PrimaryNodes 918 and 23974
[DIFF] Running subgraph Diff under PrimaryNodes 10400 and 33456
[DIFF] Running subgraph Diff under PrimaryNodes 10966 and 34022
[DIFF] Running subgraph Diff under PrimaryNodes 10978 and 34034
[DIFF] Running subgraph Diff under PrimaryNodes 11288 and 34344
[DIFF] Running subgraph Diff under PrimaryNodes 11331 and 34387
[DIFF] Running subgraph Diff under PrimaryNodes 11375 and 34431
[DIFF] Running subgraph Diff under PrimaryNodes 11402 and 34458
[DIFF] Running subgraph Diff under PrimaryNodes 11428 and 34484
[DIFF] Running subgraph Diff under PrimaryNodes 11472 and 34528
[DIFF] Running subgraph Diff under PrimaryNodes 11483 and 3

## Generate patch

In [3]:
from PatchManager.PatchService import PatchService
from neo4j_middleware.neo4jConnector import Neo4jConnector

connector = Neo4jConnector()
connector.connect_driver()

service = PatchService()
service.load_delta('GraphDelta_init{}-updt{}.json'.format(label_init, label_updated))

patch = service.generate_DPO_patch(connector=connector)

service.save_patch_to_json(patch)

# visualize results
# update_patch.operations[0].plot_patterns()

# finally disconnect
connector.disconnect_driver()


Initialized new Connector instance.
[INFO] loading delta json....
[INFO] loading delta json: DONE.
[INFO] generate patch ....
[INFO] generate patch: DONE.
[INFO] saving patch ... 
[INFO] saving patch: DONE. 


## Apply patch

! Switch to receiver database !

optional: clear database

In [4]:
from neo4j_middleware.neo4jConnector import Neo4jConnector

# connect to neo4j
connector = Neo4jConnector()
connector.connect_driver()
print("[INFO] clear database ... ")
connector.run_cypher_statement("MATCH (n) DETACH DELETE n")
connector.disconnect_driver()
print("[INFO] clear database: DONE. ")

Initialized new Connector instance.
[INFO] clear database ... 
[INFO] clear database: DONE. 


optional: load initial model

In [5]:
from IfcGraphInterface.Ifc2GraphTranslator import IFCGraphGenerator
from neo4j_middleware.neo4jConnector import Neo4jConnector

# connect to neo4j
connector = Neo4jConnector()
connector.connect_driver()

# paths to IFC models
model_paths: List[str] = [
    './00_sampleData/IFC_stepP21/Fahrzeughalle/Fahrzeughalle.ifc',
    './00_sampleData/IFC_stepP21/Fahrzeughalle/Fahrzeughalle_updt.ifc']

graphGenerator = IFCGraphGenerator(connector, model_paths[0], None)
print('[INFO] Generating host graph... ' )
ts_host = graphGenerator.generateGraph()
print('[INFO] Generating host graph: DONE. ')
# finally disconnect
connector.disconnect_driver()

Initialized new Connector instance.
[INFO] Generating host graph... 
[IFC_P21 > ts20220510T140147 < ]: Generating graph... 
[####################################################################################################] 99% done.
[####################################################################################################] 99% done.
[####################################################################################################] 99% done.
[####################################################################################################] 99% done.
[####################################################################################################] 99% done.
[####################################################################################################] 99% done.
[####################################################################################################] 99% done.
[####################################################################################

Apply patch

In [6]:
from PatchManager.PatchService import PatchService
from neo4j_middleware.neo4jConnector import Neo4jConnector

label_init = "ts20220510T140147"
label_updated = "ts20220510T140041"

# connect to neo4j
connector = Neo4jConnector()
connector.connect_driver()

# init new PatchService object handling all load and save operations
service = PatchService()

# load patch
patch = service.load_patch_from_json('Patch_init{}-updt{}.json'.format(label_init, label_updated))

# apply the patch
print('[INFO] Applying patch ...' )
service.apply_patch(patch, connector=connector)
print('[INFO] Applying patch: DONE.' )

# finally disconnect
connector.disconnect_driver()

Initialized new Connector instance.
[INFO] loading delta json....
[INFO] loading delta json: DONE.
[INFO] Applying patch ...
[INFO] finding context...
insert push out
[INFO] Applying patch: DONE.


In [7]:
# harmonize labels
label_init = "ts20210623T091748"
label_updated = "ts20210623T091749"

connector.run_cypher_statement("MATCH (n) REMOVE n:{} SET n:{}".format(label_init, label_updated))

[]

# Parse graph back into file-based representation

In [6]:
from IfcGraphInterface.Graph2IfcTranslator import Graph2IfcTranslator
from neo4j_middleware.neo4jConnector import Neo4jConnector

# connect to neo4j
connector = Neo4jConnector()
connector.connect_driver()

label_updated = "ts20220510T140147"

print('[INFO] Parsing graph to Ifc StepP21 model... ')
generator = Graph2IfcTranslator(connector=connector, ts=label_updated)
generator.generateSPF()

path = "C:\dev\out\{}".format(label_updated)
generator.save_model(path=path)

print('[INFO] Parsing graph to Ifc StepP21 model: DONE. ')
print('[INFO] path: {}'.format(path))

# finally disconnect
connector.disconnect_driver()

Initialized new Connector instance.
[INFO] Parsing graph to Ifc StepP21 model... 
#1=IfcOpeningElement('1TGYFgbjfE2RneJqu9L8JY',$,'TU DF 1 - Rahmenstock flächenbündig:ML - 885 x 2135:2474559:1',$,$,$,$,'2474559',.OPENING.)
#2=IfcProductDefinitionShape($,$,$)
#3=IfcShapeRepresentation($,'Body','SweptSolid',$)
#4=IfcExtrudedAreaSolid($,$,$,0.46)
#5=IfcDirection((0.,0.,1.))
#6=IfcAxis2Placement3D($,$,$)
#7=IfcDirection((0.,1.,0.))
#8=IfcCartesianPoint((0.,0.,0.))
#9=IfcArbitraryClosedProfileDef(.AREA.,$,$)
#10=IfcIndexedPolyCurve($,$,.F.)
#11=IfcCartesianPointList2D(((2.315,0.885),(2.70716782324598E-16,0.885),(2.70716782324598E-16,5.41433564649196E-16),(2.315,5.41433564649196E-16),(2.315,0.885)))
#12=IfcGeometricRepresentationSubContext('Body','Model',*,*,*,*,$,$,.MODEL_VIEW.,$)
#13=IfcGeometricRepresentationContext($,'Model',3,1.E-05,$,$)
#14=IfcDirection((6.12303176911189E-17,1.))
#15=IfcAxis2Placement3D($,$,$)
#16=IfcLocalPlacement($,$)
#17=IfcAxis2Placement3D($,$,$)
#18=IfcCartesianPo

inverse patch

In [2]:
from neo4j_middleware.neo4jConnector import Neo4jConnector
from PatchManager.PatchService import PatchService

# connect to neo4j
connector = Neo4jConnector()
connector.connect_driver()

label_init = "ts20210623T091748"
label_updated = "ts20210623T091749"

# init new PatchService object handling all load and save operations
service = PatchService()

# load patch
patch = service.load_patch_from_json('Patch_init{}-updt{}.json'.format(label_init, label_updated))

# apply the patch
service.apply_patch_inverse(patch, connector=connector)

connector.disconnect_driver()

Initialized new Connector instance.
[INFO] loading delta json....
[INFO] loading delta json: DONE.
